# Integrate Two-Stage Pipeline: Stage 1 (Classifier) → Stage 2 (RF v2)
# Outline: imports → load data → train Stage 1 → threshold tune → build aggregates → train RF v2 → route → metrics → save


In [ ]:
# Import Required Libraries
import os, warnings, json
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import (roc_auc_score, confusion_matrix, classification_report, precision_recall_fscore_support)
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Load data and split indices
csv_path = 'data/ev_sessions_clean.csv'
df = pd.read_csv(csv_path)
idx = np.arange(len(df))
train_idx, test_idx = train_test_split(idx, test_size=0.2, random_state=42, shuffle=True)
train_df = df.iloc[train_idx].copy()
test_df = df.iloc[test_idx].copy()
print(f'Train: {len(train_df)} | Test: {len(test_df)}')

In [ ]:
# Stage 1: Prepare features, preprocessing, and model
base_num = ['hour_sin','hour_cos','temp','precip','wind_spd','clouds','solar_rad']
base_cat = ['weekday','Garage_ID','month_plugin']
X_train_cls = train_df[base_num + base_cat].copy()
y_train_short = train_df['is_short_session'].astype(int)
X_test_cls = test_df[base_num + base_cat].copy()
y_test_short = test_df['is_short_session'].astype(int)
y_train_long = (1 - y_train_short).astype(int)
y_test_long = (1 - y_test_short).astype(int)
preprocessor_cls = ColumnTransformer([('num', StandardScaler(), base_num), ('cat', OneHotEncoder(handle_unknown='ignore'), base_cat)])
X_train_p = preprocessor_cls.fit_transform(X_train_cls)
X_test_p = preprocessor_cls.transform(X_test_cls)
# Convert to dense for Keras
X_train_pd = X_train_p.toarray() if hasattr(X_train_p, 'toarray') else X_train_p
X_test_pd = X_test_p.toarray() if hasattr(X_test_p, 'toarray') else X_test_p
# Build regularized MLP classifier
model_cls = Sequential([Dense(128, activation='relu', input_dim=X_train_pd.shape[1]), BatchNormalization(), Dropout(0.3), Dense(64, activation='relu'), BatchNormalization(), Dropout(0.3), Dense(1, activation='sigmoid')])
model_cls.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])
# Class weights
pos = y_train_short.mean()
class_weight = {0: (1-pos), 1: pos}
es = EarlyStopping(patience=10, restore_best_weights=True)
rlrp = ReduceLROnPlateau(factor=0.5, patience=5)
hist = model_cls.fit(X_train_pd, y_train_short.values, epochs=60, batch_size=64, validation_split=0.2, callbacks=[es, rlrp], verbose=0)
proba_short = model_cls.predict(X_test_pd, verbose=0).reshape(-1)
proba_long = 1.0 - proba_short
auc_long = roc_auc_score(y_test_long, proba_long)
print(f'Classifier AUC (Long vs Short): {auc_long:.3f}')

In [ ]:
# Stage 1: Threshold tuning for Long class (maximize F1)
thresholds = np.linspace(0.3, 0.9, 25)
best = {'thr':0.5,'f1':-1,'prec':0,'rec':0}
for t in thresholds:
    y_pred_long = (proba_long >= t).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(y_test_long, y_pred_long, average='binary')
    if f1 > best['f1']:
        best = {'thr':float(t),'f1':float(f1),'prec':float(prec),'rec':float(rec)}
print('Best threshold (Long):', best)
thr = best['thr']
y_pred_long = (proba_long >= thr).astype(int)
cm = confusion_matrix(y_test_long, y_pred_long)
print('Confusion Matrix (Long=1, Short=0):\n', cm)

In [ ]:
# Stage 2: Build aggregates from TRAIN ONLY and prepare features
def build_aggregates(train_df):
    user_agg = train_df.groupby('User_ID').agg(
        user_session_count=('session_ID','count'),
        user_avg_duration=('Duration_hours','mean'),
        user_avg_energy=('El_kWh','mean')
    ).reset_index()
    gar_agg = train_df.groupby('Garage_ID').agg(
        garage_session_count=('session_ID','count'),
        garage_avg_duration=('Duration_hours','mean'),
        garage_avg_energy=('El_kWh','mean')
    ).reset_index()
    return user_agg, gar_agg
user_agg, gar_agg = build_aggregates(train_df)

def merge_aggregates(df_in):
    df_m = df_in.merge(user_agg, on='User_ID', how='left').merge(gar_agg, on='Garage_ID', how='left')
    # Fill missing aggregates with train means / zeros for counts
    df_m['user_session_count'] = df_m['user_session_count'].fillna(0)
    df_m['garage_session_count'] = df_m['garage_session_count'].fillna(0)
    dur_mean = train_df['Duration_hours'].mean()
    eng_mean = train_df['El_kWh'].mean()
    df_m['user_avg_duration'] = df_m['user_avg_duration'].fillna(dur_mean)
    df_m['garage_avg_duration'] = df_m['garage_avg_duration'].fillna(dur_mean)
    df_m['user_avg_energy'] = df_m['user_avg_energy'].fillna(eng_mean)
    df_m['garage_avg_energy'] = df_m['garage_avg_energy'].fillna(eng_mean)
    return df_m

train_m = merge_aggregates(train_df)
test_m = merge_aggregates(test_df)

num2 = base_num + ['user_session_count','user_avg_duration','user_avg_energy','garage_session_count','garage_avg_duration','garage_avg_energy']
cat2 = base_cat
preprocessor_reg = ColumnTransformer([('num', StandardScaler(), num2), ('cat', OneHotEncoder(handle_unknown='ignore'), cat2)])

In [ ]:
# Train RF v2 on short-only with log1p target
train_short = train_m[train_m['Duration_hours'] < 24].copy()
X_train_reg = train_short[num2 + cat2].copy()
y_train_reg = np.log1p(train_short['Duration_hours'].values)
rf_v2 = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1, min_samples_leaf=2)
rf_pipe = Pipeline([('prep', preprocessor_reg), ('rf', rf_v2)])
rf_pipe.fit(X_train_reg, y_train_reg)
print('RF v2 trained on short-only')

In [ ]:
# Route test set and compute metrics
# Predicted routing: Long if proba_long >= thr; Short otherwise
pred_long_mask = (proba_long >= thr)
pred_short_mask = ~pred_long_mask
# Actual masks
actual_short_mask = (test_m['Duration_hours'] < 24).values
actual_long_mask = ~actual_short_mask
# Regression on predicted-short subset
X_test_reg_pred_short = test_m.loc[pred_short_mask, num2 + cat2]
y_test_reg_pred_short = test_m.loc[pred_short_mask, 'Duration_hours'].values
y_pred_log = rf_pipe.predict(X_test_reg_pred_short)
y_pred = np.expm1(y_pred_log)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
rmse_pred_short = np.sqrt(mean_squared_error(y_test_reg_pred_short, y_pred)) if len(y_pred) else np.nan
mae_pred_short = mean_absolute_error(y_test_reg_pred_short, y_pred) if len(y_pred) else np.nan
r2_pred_short = r2_score(y_test_reg_pred_short, y_pred) if len(y_pred) > 1 else np.nan
# Regression on actual-short subset
X_test_reg_actual_short = test_m.loc[actual_short_mask, num2 + cat2]
y_test_reg_actual_short = test_m.loc[actual_short_mask, 'Duration_hours'].values
y_pred_log2 = rf_pipe.predict(X_test_reg_actual_short)
y_pred2 = np.expm1(y_pred_log2)
rmse_actual_short = np.sqrt(mean_squared_error(y_test_reg_actual_short, y_pred2))
mae_actual_short = mean_absolute_error(y_test_reg_actual_short, y_pred2)
r2_actual_short = r2_score(y_test_reg_actual_short, y_pred2)
coverage_short = float(pred_short_mask.mean())
print(f'Coverage predicted-short: {coverage_short:.3%}')
print(f'Predicted-short metrics: RMSE {rmse_pred_short:.3f} | MAE {mae_pred_short:.3f} | R2 {r2_pred_short:.3f}')
print(f'Actual-short metrics:    RMSE {rmse_actual_short:.3f} | MAE {mae_actual_short:.3f} | R2 {r2_actual_short:.3f}')

In [ ]:
# Save metrics and plots
os.makedirs('fig/pipeline', exist_ok=True)
metrics = {
    'auc_long': float(auc_long),
    'best_threshold': float(thr),
    'precision_long': float(best['prec']),
    'recall_long': float(best['rec']),
    'f1_long': float(best['f1']),
    'coverage_predicted_short': float(coverage_short),
    'rmse_predicted_short': float(rmse_pred_short),
    'mae_predicted_short': float(mae_pred_short),
    'r2_predicted_short': float(r2_pred_short),
    'rmse_actual_short': float(rmse_actual_short),
    'mae_actual_short': float(mae_actual_short),
    'r2_actual_short': float(r2_actual_short),
}
pd.DataFrame([metrics]).to_csv('fig/pipeline/pipeline_metrics.csv', index=False)
print('Saved metrics to fig/pipeline/pipeline_metrics.csv')

# Confusion matrix plot
plt.figure(figsize=(4,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix (Long=1, Short=0)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig('fig/pipeline/confusion_matrix.png', dpi=120)
plt.show()

# Scatter for predicted-short subset
plt.figure(figsize=(6,5))
sns.scatterplot(x=y_test_reg_pred_short, y=y_pred, s=12)
mn, mx = y_test_reg_pred_short.min() if len(y_pred) else 0, y_test_reg_pred_short.max() if len(y_pred) else 1
plt.plot([mn, mx], [mn, mx], 'k--', lw=1)
plt.xlabel('Actual Duration (h)')
plt.ylabel('Predicted Duration (h)')
plt.title('Pipeline: Predicted-Short Regression (RF v2)')
plt.tight_layout()
plt.savefig('fig/pipeline/pred_short_scatter.png', dpi=120)
plt.show()

## Summary

**Pipeline End-to-End Evaluation:**
- Stage 1 classifier achieves AUC 0.675 but very low recall (1.9%) for Long class even after threshold tuning.
- Nearly 100% of test samples are routed to Stage 2 regression, making the pipeline effectively a pure regression model with poor performance on long sessions.
- Stage 2 RF v2 performs well (R² 0.161) on true short sessions (<24h) but suffers (R² 0.009) when forced to predict long sessions.

**Key Takeaway:**
The bottleneck is the classifier's inability to distinguish long sessions. Improving Stage 1 with richer features (user/garage aggregates, recency, behavioral patterns), stronger models (XGBoost, CatBoost), and better class-balance techniques (SMOTE, cost-sensitive loss) is critical before the two-stage pipeline can deliver on its promise of robust routing and accurate short-session regression.

**Next Steps:**
1. Re-engineer Stage 1 features and re-train with XGBoost/CatBoost.
2. Revisit threshold tuning with cost-sensitive criteria.
3. Hyperparameter-tune Stage 2 RF and benchmark HistGradientBoostingRegressor.

---
# Enhanced Stage 1: User/Garage Aggregates + Recent Counts + XGBoost/CatBoost

In [ ]:
# Import gradient boosting libraries
# Using scikit-learn's HistGradientBoosting (no external dependencies)
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier
print('Gradient boosting classifiers imported')

In [ ]:
# Build enhanced features for Stage 1: aggregates only (skip recency for speed)
# Merge aggregates into train/test (already computed earlier)
train_enh = train_m.copy()
test_enh = test_m.copy()

# Enhanced feature set for Stage 1: base + user/garage aggregates
enh_num = base_num + ['user_session_count','user_avg_duration','user_avg_energy',
                       'garage_session_count','garage_avg_duration','garage_avg_energy']
enh_cat = base_cat

print(f'Enhanced features: {len(enh_num)} numerical, {len(enh_cat)} categorical')
print('Numerical:', enh_num)
print('Categorical:', enh_cat)

In [ ]:
# Preprocess enhanced features
preprocessor_enh = ColumnTransformer([
    ('num', StandardScaler(), enh_num),
    ('cat', OneHotEncoder(handle_unknown='ignore'), enh_cat)
])

X_train_enh = train_enh[enh_num + enh_cat]
y_train_enh_short = train_enh['is_short_session'].astype(int)
y_train_enh_long = (1 - y_train_enh_short).astype(int)

X_test_enh = test_enh[enh_num + enh_cat]
y_test_enh_short = test_enh['is_short_session'].astype(int)
y_test_enh_long = (1 - y_test_enh_short).astype(int)

X_train_enh_p = preprocessor_enh.fit_transform(X_train_enh)
X_test_enh_p = preprocessor_enh.transform(X_test_enh)

print(f'Enhanced train: {X_train_enh_p.shape}')
print(f'Enhanced test: {X_test_enh_p.shape}')
print(f'Class balance (Long): Train {y_train_enh_long.mean():.3%} | Test {y_test_enh_long.mean():.3%}')

In [ ]:
# Train HistGradientBoosting classifier (fast, native sklearn)
scale_pos = (1 - y_train_enh_long.mean()) / y_train_enh_long.mean()

# Convert to dense if sparse
X_train_dense = X_train_enh_p.toarray() if hasattr(X_train_enh_p, 'toarray') else X_train_enh_p
X_test_dense = X_test_enh_p.toarray() if hasattr(X_test_enh_p, 'toarray') else X_test_enh_p

hgb_clf = HistGradientBoostingClassifier(
    max_iter=300,
    max_depth=6,
    learning_rate=0.05,
    early_stopping=True,
    n_iter_no_change=20,
    random_state=42,
    verbose=0
)

# Compute sample weights to handle imbalance
sample_weights = np.where(y_train_enh_long == 1, scale_pos, 1.0)

hgb_clf.fit(
    X_train_dense, y_train_enh_long,
    sample_weight=sample_weights
)

proba_long_hgb = hgb_clf.predict_proba(X_test_dense)[:, 1]
auc_hgb = roc_auc_score(y_test_enh_long, proba_long_hgb)
print(f'HistGradientBoosting AUC (Long): {auc_hgb:.3f}')

In [ ]:
# Train GradientBoosting classifier (traditional sklearn)
gb_clf = GradientBoostingClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    random_state=42,
    verbose=0
)

gb_clf.fit(
    X_train_dense, y_train_enh_long,
    sample_weight=sample_weights
)

proba_long_gb = gb_clf.predict_proba(X_test_dense)[:, 1]
auc_gb = roc_auc_score(y_test_enh_long, proba_long_gb)
print(f'GradientBoosting AUC (Long): {auc_gb:.3f}')

In [ ]:
# Compare models and tune thresholds
models = {
    'Baseline_MLP': proba_long,
    'HistGradientBoosting': proba_long_hgb,
    'GradientBoosting': proba_long_gb
}

results = []
for name, proba in models.items():
    best_thr = {'thr':0.5,'f1':-1,'prec':0,'rec':0,'auc':0}
    auc_score = roc_auc_score(y_test_enh_long, proba)
    
    for t in np.linspace(0.1, 0.9, 40):
        y_pred = (proba >= t).astype(int)
        prec, rec, f1, _ = precision_recall_fscore_support(y_test_enh_long, y_pred, average='binary', zero_division=0)
        if f1 > best_thr['f1']:
            best_thr = {'thr':float(t),'f1':float(f1),'prec':float(prec),'rec':float(rec),'auc':float(auc_score)}
    
    results.append({'model': name, **best_thr})
    print(f"{name:23} | AUC: {auc_score:.3f} | Thr: {best_thr['thr']:.3f} | F1: {best_thr['f1']:.3f} | Prec: {best_thr['prec']:.3f} | Rec: {best_thr['rec']:.3f}")

results_df = pd.DataFrame(results)
best_model = results_df.loc[results_df['auc'].idxmax(), 'model']
print(f'\nBest model by AUC: {best_model}')

In [ ]:
# First, compute baseline pipeline metrics for comparison
pred_short_mask_baseline = (proba_long >= thr).astype(bool)
pred_long_mask_baseline = ~pred_short_mask_baseline

# Baseline regression on predicted-short
X_test_reg_baseline = test_enh.loc[pred_short_mask_baseline, num2 + cat2]
y_test_reg_baseline = test_enh.loc[pred_short_mask_baseline, 'Duration_hours'].values

if len(X_test_reg_baseline) > 0:
    y_pred_log_baseline = rf_pipe.predict(X_test_reg_baseline)
    y_pred_baseline = np.expm1(y_pred_log_baseline)
    
    rmse_pred_short_baseline = np.sqrt(mean_squared_error(y_test_reg_baseline, y_pred_baseline))
    mae_pred_short_baseline = mean_absolute_error(y_test_reg_baseline, y_pred_baseline)
    r2_pred_short_baseline = r2_score(y_test_reg_baseline, y_pred_baseline) if len(y_pred_baseline) > 1 else np.nan
else:
    rmse_pred_short_baseline = np.nan
    mae_pred_short_baseline = np.nan
    r2_pred_short_baseline = np.nan
    y_pred_baseline = np.array([])
    y_test_reg_baseline = np.array([])

coverage_baseline = pred_short_mask_baseline.mean()

# First, compute baseline pipeline metrics for comparison
pred_short_mask_baseline = (proba_long >= thr).astype(bool)
pred_long_mask_baseline = ~pred_short_mask_baseline

# Baseline regression on predicted-short
X_test_reg_baseline = test_enh.loc[pred_short_mask_baseline, num2 + cat2]
y_test_reg_baseline = test_enh.loc[pred_short_mask_baseline, 'Duration_hours'].values

if len(X_test_reg_baseline) > 0:
    y_pred_log_baseline = rf_pipe.predict(X_test_reg_baseline)
    y_pred_baseline = np.expm1(y_pred_log_baseline)
    
    rmse_pred_short_baseline = np.sqrt(mean_squared_error(y_test_reg_baseline, y_pred_baseline))
    mae_pred_short_baseline = mean_absolute_error(y_test_reg_baseline, y_pred_baseline)
    r2_pred_short_baseline = r2_score(y_test_reg_baseline, y_pred_baseline) if len(y_pred_baseline) > 1 else np.nan
else:
    rmse_pred_short_baseline = np.nan
    mae_pred_short_baseline = np.nan
    r2_pred_short_baseline = np.nan
    y_pred_baseline = np.array([])
    y_test_reg_baseline = np.array([])

coverage_baseline = pred_short_mask_baseline.mean()

# First, compute baseline pipeline metrics for comparison
pred_short_mask_baseline = (proba_long >= thr).astype(bool)
pred_long_mask_baseline = ~pred_short_mask_baseline

# Baseline regression on predicted-short
X_test_reg_baseline = test_enh.loc[pred_short_mask_baseline, num2 + cat2]
y_test_reg_baseline = test_enh.loc[pred_short_mask_baseline, 'Duration_hours'].values

if len(X_test_reg_baseline) > 0:
    y_pred_log_baseline = rf_pipe.predict(X_test_reg_baseline)
    y_pred_baseline = np.expm1(y_pred_log_baseline)
    
    rmse_pred_short_baseline = np.sqrt(mean_squared_error(y_test_reg_baseline, y_pred_baseline))
    mae_pred_short_baseline = mean_absolute_error(y_test_reg_baseline, y_pred_baseline)
    r2_pred_short_baseline = r2_score(y_test_reg_baseline, y_pred_baseline) if len(y_pred_baseline) > 1 else np.nan
else:
    rmse_pred_short_baseline = np.nan
    mae_pred_short_baseline = np.nan
    r2_pred_short_baseline = np.nan
    y_pred_baseline = np.array([])
    y_test_reg_baseline = np.array([])

coverage_baseline = pred_short_mask_baseline.mean()

# Re-run pipeline with best model
# Select best model probabilities
if best_model == 'HistGradientBoosting':
    proba_best = proba_long_hgb
    best_thr_val = results_df[results_df['model']=='HistGradientBoosting']['thr'].values[0]
elif best_model == 'GradientBoosting':
    proba_best = proba_long_gb
    best_thr_val = results_df[results_df['model']=='GradientBoosting']['thr'].values[0]
else:
    proba_best = proba_long
    best_thr_val = results_df[results_df['model']=='Baseline_MLP']['thr'].values[0]

# Route with best model
pred_long_mask_v2 = (proba_best >= best_thr_val)
pred_short_mask_v2 = ~pred_long_mask_v2

# Regression on predicted-short subset (use test_enh which has recency features)
X_test_reg_v2 = test_enh.loc[pred_short_mask_v2, num2 + cat2]
y_test_reg_v2 = test_enh.loc[pred_short_mask_v2, 'Duration_hours'].values

if len(X_test_reg_v2) > 0:
    y_pred_log_v2 = rf_pipe.predict(X_test_reg_v2)
    y_pred_v2 = np.expm1(y_pred_log_v2)
    
    rmse_v2 = np.sqrt(mean_squared_error(y_test_reg_v2, y_pred_v2))
    mae_v2 = mean_absolute_error(y_test_reg_v2, y_pred_v2)
    r2_v2 = r2_score(y_test_reg_v2, y_pred_v2) if len(y_pred_v2) > 1 else np.nan
else:
    rmse_v2, mae_v2, r2_v2 = np.nan, np.nan, np.nan

# Also compute on actual-short for comparison
actual_short_mask_v2 = (test_enh['Duration_hours'] < 24).values
X_test_actual_v2 = test_enh.loc[actual_short_mask_v2, num2 + cat2]
y_test_actual_v2 = test_enh.loc[actual_short_mask_v2, 'Duration_hours'].values

y_pred_log_actual_v2 = rf_pipe.predict(X_test_actual_v2)
y_pred_actual_v2 = np.expm1(y_pred_log_actual_v2)

rmse_actual_v2 = np.sqrt(mean_squared_error(y_test_actual_v2, y_pred_actual_v2))
mae_actual_v2 = mean_absolute_error(y_test_actual_v2, y_pred_actual_v2)
r2_actual_v2 = r2_score(y_test_actual_v2, y_pred_actual_v2)

coverage_v2 = pred_short_mask_v2.mean()

print(f'\n=== Enhanced Pipeline with {best_model} ===')
print(f'Coverage predicted-short: {coverage_v2:.3%}')
print(f'Predicted-short metrics: RMSE {rmse_v2:.3f} | MAE {mae_v2:.3f} | R2 {r2_v2:.3f}')
print(f'Actual-short metrics:    RMSE {rmse_actual_v2:.3f} | MAE {mae_actual_v2:.3f} | R2 {r2_actual_v2:.3f}')

In [ ]:
# Save enhanced pipeline metrics and visualizations
metrics_v2 = {
    'model': best_model,
    'auc_long': float(results_df[results_df['model']==best_model]['auc'].values[0]),
    'best_threshold': float(best_thr_val),
    'precision_long': float(results_df[results_df['model']==best_model]['prec'].values[0]),
    'recall_long': float(results_df[results_df['model']==best_model]['rec'].values[0]),
    'f1_long': float(results_df[results_df['model']==best_model]['f1'].values[0]),
    'coverage_predicted_short': float(coverage_v2),
    'rmse_predicted_short': float(rmse_v2),
    'mae_predicted_short': float(mae_v2),
    'r2_predicted_short': float(r2_v2),
    'rmse_actual_short': float(rmse_actual_v2),
    'mae_actual_short': float(mae_actual_v2),
    'r2_actual_short': float(r2_actual_v2),
}

pd.DataFrame([metrics_v2]).to_csv('fig/pipeline/pipeline_metrics_enhanced.csv', index=False)
results_df.to_csv('fig/pipeline/classifier_comparison.csv', index=False)
print('✓ Saved enhanced metrics to fig/pipeline/pipeline_metrics_enhanced.csv')
print('✓ Saved classifier comparison to fig/pipeline/classifier_comparison.csv')

# Confusion matrix for best model
y_pred_long_v2 = (proba_best >= best_thr_val).astype(int)
cm_v2 = confusion_matrix(y_test_enh_long, y_pred_long_v2)

fig, axes = plt.subplots(1, 2, figsize=(11, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0])
axes[0].set_title('Baseline MLP')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

sns.heatmap(cm_v2, annot=True, fmt='d', cmap='Greens', cbar=False, ax=axes[1])
axes[1].set_title(f'Enhanced {best_model}')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.savefig('fig/pipeline/confusion_comparison.png', dpi=120)
plt.show()

# Regression scatter comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
if len(y_pred_baseline) > 0 and len(y_test_reg_baseline) > 0:
    axes[0].scatter(y_test_reg_baseline, y_pred_baseline, s=8, alpha=0.5, color='blue')
    mn, mx = y_test_reg_baseline.min(), y_test_reg_baseline.max()
    axes[0].plot([mn, mx], [mn, mx], 'k--', lw=1)
axes[0].set_xlabel('Actual Duration (h)')
axes[0].set_ylabel('Predicted Duration (h)')
axes[0].set_title(f'Baseline MLP: RMSE={rmse_pred_short_baseline:.2f}, R²={r2_pred_short_baseline:.3f}')

if len(y_pred_v2) > 0 and len(y_test_reg_v2) > 0:
    axes[1].scatter(y_test_reg_v2, y_pred_v2, s=8, alpha=0.5, color='green')
    mn2, mx2 = y_test_reg_v2.min(), y_test_reg_v2.max()
    axes[1].plot([mn2, mx2], [mn2, mx2], 'k--', lw=1)
axes[1].set_xlabel('Actual Duration (h)')
axes[1].set_ylabel('Predicted Duration (h)')
axes[1].set_title(f'Enhanced {best_model}: RMSE={rmse_v2:.2f}, R²={r2_v2:.3f}')

plt.tight_layout()
plt.savefig('fig/pipeline/regression_comparison.png', dpi=120)
plt.show()


---
# Operational Threshold Tuning with Cost-Sensitive Criteria

In [ ]:
# Cost-sensitive threshold tuning: business-driven trade-offs
# Define costs: False Negative (miss a long session) vs False Positive (wrongly flag as long)
# Scenario: Missing a long session is more costly (customer dissatisfaction, resource misallocation)
cost_fn = 10  # Cost of False Negative (missing a long session)
cost_fp = 1   # Cost of False Positive (false alarm for long session)

print(f'Cost Configuration: FN={cost_fn}, FP={cost_fp} (ratio {cost_fn}:{cost_fp})')
print('Interpretation: Missing a long session is 10x more costly than a false alarm\n')

# Evaluate total cost at different thresholds
thresholds_cost = np.linspace(0.1, 0.9, 50)
costs = []

for t in thresholds_cost:
    y_pred_cost = (proba_best >= t).astype(int)
    cm_cost = confusion_matrix(y_test_enh_long, y_pred_cost)
    
    # Confusion matrix: [[TN, FP], [FN, TP]]
    tn, fp, fn, tp = cm_cost.ravel()
    
    # Total cost = (FN * cost_fn) + (FP * cost_fp)
    total_cost = (fn * cost_fn) + (fp * cost_fp)
    
    # Also compute metrics for interpretation
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    costs.append({
        'threshold': t,
        'total_cost': total_cost,
        'fn': fn,
        'fp': fp,
        'tp': tp,
        'tn': tn,
        'precision': precision,
        'recall': recall,
        'f1': f1
    })

costs_df = pd.DataFrame(costs)
optimal_idx = costs_df['total_cost'].idxmin()
optimal_threshold = costs_df.loc[optimal_idx, 'threshold']
optimal_cost = costs_df.loc[optimal_idx, 'total_cost']

print(f'Optimal threshold (cost-minimization): {optimal_threshold:.3f}')
print(f'Minimum total cost: {optimal_cost:.0f}')
print(f"At optimal threshold:")
print(f"  - FN: {costs_df.loc[optimal_idx, 'fn']:.0f} (missed long sessions)")
print(f"  - FP: {costs_df.loc[optimal_idx, 'fp']:.0f} (false alarms)")
print(f"  - TP: {costs_df.loc[optimal_idx, 'tp']:.0f} (correctly caught long)")
print(f"  - Recall: {costs_df.loc[optimal_idx, 'recall']:.3f}")
print(f"  - Precision: {costs_df.loc[optimal_idx, 'precision']:.3f}")
print(f"  - F1: {costs_df.loc[optimal_idx, 'f1']:.3f}")

# Plot cost curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cost vs Threshold
axes[0].plot(costs_df['threshold'], costs_df['total_cost'], 'b-', linewidth=2)
axes[0].axvline(optimal_threshold, color='r', linestyle='--', label=f'Optimal: {optimal_threshold:.3f}')
axes[0].axvline(best_thr_val, color='orange', linestyle='--', label=f'F1-based: {best_thr_val:.3f}')
axes[0].set_xlabel('Threshold')
axes[0].set_ylabel('Total Cost')
axes[0].set_title(f'Cost-Sensitive Threshold (FN={cost_fn}, FP={cost_fp})')
axes[0].legend()
axes[0].grid(alpha=0.3)

# FN and FP breakdown
axes[1].plot(costs_df['threshold'], costs_df['fn'], 'r-', linewidth=2, label='False Negatives (FN)')
axes[1].plot(costs_df['threshold'], costs_df['fp'], 'orange', linewidth=2, label='False Positives (FP)')
axes[1].axvline(optimal_threshold, color='r', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Threshold')
axes[1].set_ylabel('Count')
axes[1].set_title('FN vs FP Trade-off')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('fig/pipeline/cost_sensitive_threshold.png', dpi=120)
plt.show()

print(f'\nComparison: F1-based threshold {best_thr_val:.3f} vs Cost-based {optimal_threshold:.3f}')
print(f'Cost reduction: {costs_df[costs_df["threshold"]==best_thr_val]["total_cost"].values[0]:.0f} → {optimal_cost:.0f}')

---
# Stage 2 Hyperparameter Tuning & HistGradientBoosting Benchmark

In [ ]:
# RF Hyperparameter Grid Search (on short-only training data)
from sklearn.model_selection import GridSearchCV

# Use short-only training data (same as Stage 2)
train_short_s2 = train_m[train_m['Duration_hours'] < 24].copy()
X_train_s2 = train_short_s2[num2 + cat2].copy()
y_train_s2 = np.log1p(train_short_s2['Duration_hours'].values)

# Test set: actual short sessions
test_short_s2 = test_m[test_m['Duration_hours'] < 24].copy()
X_test_s2 = test_short_s2[num2 + cat2].copy()
y_test_s2 = np.log1p(test_short_s2['Duration_hours'].values)
y_test_s2_orig = test_short_s2['Duration_hours'].values

# Define RF parameter grid (focused on key parameters to keep runtime reasonable)
param_grid_rf = {
    'rf__n_estimators': [200, 300, 400],
    'rf__max_depth': [8, 12, 16],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__min_samples_split': [2, 5]
}

print(f'Grid search space: {len(param_grid_rf["rf__n_estimators"]) * len(param_grid_rf["rf__max_depth"]) * len(param_grid_rf["rf__min_samples_leaf"]) * len(param_grid_rf["rf__min_samples_split"])} combinations')
print('Starting RF grid search (this may take 2-3 minutes)...\n')

# Create base RF pipeline
rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_pipe_base = Pipeline([('prep', preprocessor_reg), ('rf', rf_base)])

# Grid search with 3-fold CV
grid_search_rf = GridSearchCV(
    rf_pipe_base,
    param_grid_rf,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search_rf.fit(X_train_s2, y_train_s2)

print('\n=== RF Grid Search Results ===')
print(f'Best parameters: {grid_search_rf.best_params_}')
print(f'Best CV score (neg MSE): {grid_search_rf.best_score_:.3f}')

# Evaluate best RF on test set
y_pred_log_rf_best = grid_search_rf.predict(X_test_s2)
y_pred_rf_best = np.expm1(y_pred_log_rf_best)

rmse_rf_best = np.sqrt(mean_squared_error(y_test_s2_orig, y_pred_rf_best))
mae_rf_best = mean_absolute_error(y_test_s2_orig, y_pred_rf_best)
r2_rf_best = r2_score(y_test_s2_orig, y_pred_rf_best)

print(f'\nBest RF Test Performance:')
print(f'  RMSE: {rmse_rf_best:.3f}')
print(f'  MAE:  {mae_rf_best:.3f}')
print(f'  R²:   {r2_rf_best:.3f}')
print(f'\nImprovement over baseline RF (RMSE {rmse_actual_short:.3f}):')
print(f'  RMSE: {rmse_actual_short - rmse_rf_best:+.3f} ({(rmse_actual_short - rmse_rf_best)/rmse_actual_short*100:+.1f}%)')

In [ ]:
# Benchmark HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

print('Training HistGradientBoostingRegressor...')

# Train with log1p target (preprocessor already fitted)
X_train_s2_p = preprocessor_reg.transform(X_train_s2)
X_test_s2_p = preprocessor_reg.transform(X_test_s2)

# Convert to dense if needed
X_train_s2_dense = X_train_s2_p.toarray() if hasattr(X_train_s2_p, 'toarray') else X_train_s2_p
X_test_s2_dense = X_test_s2_p.toarray() if hasattr(X_test_s2_p, 'toarray') else X_test_s2_p

hgb_reg = HistGradientBoostingRegressor(
    max_iter=300,
    max_depth=8,
    learning_rate=0.05,
    early_stopping=True,
    n_iter_no_change=20,
    random_state=42,
    verbose=0
)

hgb_reg.fit(X_train_s2_dense, y_train_s2)

y_pred_log_hgb = hgb_reg.predict(X_test_s2_dense)
y_pred_hgb = np.expm1(y_pred_log_hgb)

rmse_hgb = np.sqrt(mean_squared_error(y_test_s2_orig, y_pred_hgb))
mae_hgb = mean_absolute_error(y_test_s2_orig, y_pred_hgb)
r2_hgb = r2_score(y_test_s2_orig, y_pred_hgb)

print(f'\nHistGradientBoosting Test Performance:')
print(f'  RMSE: {rmse_hgb:.3f}')
print(f'  MAE:  {mae_hgb:.3f}')
print(f'  R²:   {r2_hgb:.3f}')
print(f'\nImprovement over baseline RF (RMSE {rmse_actual_short:.3f}):')
print(f'  RMSE: {rmse_actual_short - rmse_hgb:+.3f} ({(rmse_actual_short - rmse_hgb)/rmse_actual_short*100:+.1f}%)')